In [7]:
import FinanceDataReader as fdr
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame, concat
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from openpyxl import load_workbook
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta

In [39]:
# 현재 일자
today = datetime.now()
date2 = today.strftime('%Y%m%d') 
# 1일 전 일자
yesterday = today - timedelta(days=1)
date =  yesterday.strftime('%Y%m%d')

'20250121'

# WICS 대분류,중분류 매핑-https://www.wiseindex.com/About/WICS
# http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20241227&sec_cd=G1010페이지가 소분류까지는 못가져옴

In [11]:
wics_lc = {
    10: '에너지',
    15: '소재',
    20: '산업재',
    25: '경기관련소비재',
    30: '필수소비재',
    35: '건강관리',
    40: '금융',
    45: 'IT',
    50: '커뮤니케이션서비스',
    55: '유틸리티'
}

wics_mc = {
    1010: '에너지',
    1510: '소재',
    2010: '자본재',
    2020: '상업서비스와공급품',
    2030: '운송',
    2510: '자동차와부품',
    2520: '내구소비재와의류',
    2530: '호텔,레스토랑,레저 등',
    2550: '소매(유통)',
    2560: '교육서비스',
    3010: '식품과기본식료품소매',
    3020: '식품,음료,담배',
    3030: '가정용품과개인용품',
    3510: '건강관리장비와서비스',
    3520: '제약과생물공학',
    4010: '은행',
    4020: '증권',
    4030: '다각화된금융',
    4040: '보험',
    4050: '부동산',
    4510: '소프트웨어와서비스',
    4520: '기술하드웨어와장비',
    4530: '반도체와반도체장비',
    4535: '전자와 전기제품',
    4540: '디스플레이',
    5010: '전기통신서비스',
    5020: '미디어와엔터테인먼트',
    5510: '유틸리티'
}


# URL 생성 함수
def wics_url(date, wics_code):
    '''
    파라미터:
    - date[str]: 데이터에 해당하는 날짜 (형식: yyyymmdd)
    - wics_code[int]: WICS 코드 (wics_lc 또는 wics_mc의 키 값)

    반환값:
    - url[str]: 요청에 사용할 URL
    '''
    url = 'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&' \
          'dt=' + date + '&sec_cd=G' + str(wics_code)
    return url

# 메인 코드
if __name__ == '__main__':
    # 결과를 저장할 데이터프레임 초기화
    df = DataFrame(columns=['code', 'name', 'ls', 'ms'])

    # WICS 중분류 코드를 사용해 데이터 수집
    for wics_code in wics_mc.keys():
        response = requests.get(wics_url(date, wics_code))

        if response.status_code == 200:  # 요청 성공
            json_list = response.json()  # JSON 응답을 파이썬 딕셔너리로 변환

            # 리스트에서 필요한 정보 추출
            temp_df = DataFrame(
                [{
                    'code': json['CMP_CD'],  # 회사 코드
                    'name': json['CMP_KOR'],  # 회사 이름 (한글)
                    'ls': json['SEC_NM_KOR'],  # 대분류
                    'ms': json['IDX_NM_KOR'][5:]  # 중분류
                } for json in json_list['list']]
            )
            # 기존 데이터프레임과 병합
            df = concat([df, temp_df], ignore_index=True)
        else:
            # 요청 실패 시 에러 메시지 출력
            print('Error:', response.status_code)
            print('WICS 코드:', wics_code)

    # 코드 기준으로 데이터프레임 정렬
    df = df.sort_values(by='code', ascending=True).reset_index(drop=True)

    # 열 이름 변경
    df.rename(columns={'code': 'Code', 'name': 'Name', 'ls': 'Ls', 'ms': 'Ms'}, inplace=True)

    # 결과 데이터프레임 출력
    print(df.head())

     Code   Name       Ls        Ms
0  000020   동화약품     건강관리   제약과생물공학
1  000040  KR모터스  경기관련소비재    자동차와부품
2  000050     경방  경기관련소비재  내구소비재와의류
3  000070  삼양홀딩스       소재        소재
4  000080  하이트진로    필수소비재  식품,음료,담배


In [13]:
df_stocks = fdr.StockListing('KRX') # KRX: 2,663 종목(=코스피+코스닥+코넥스)

In [14]:
# 'Code' 기준으로 df와 df_stocks 병합
df_merged = pd.merge(df, df_stocks, on='Code', how='inner')

In [15]:
df_merged

,Code,Name_x,Ls,Ms,ISU_CD,Name_y,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,000020,동화약품,건강관리,제약과생물공학,KR7000020008,동화약품,KOSPI,,6260,2,-80,-1.26,6370,6380,6200,68785,430532880,174851002200,27931470,STK
1,000040,KR모터스,경기관련소비재,자동차와부품,KR7000040006,KR모터스,KOSPI,,450,2,-2,-0.44,449,461,447,91684,41493372,27059790600,60132868,STK
2,000050,경방,경기관련소비재,내구소비재와의류,KR7000050005,경방,KOSPI,,6480,2,-30,-0.46,6730,6730,6480,2364,15670670,177650949600,27415270,STK
3,000070,삼양홀딩스,소재,소재,KR7000070003,삼양홀딩스,KOSPI,,64000,1,900,1.43,63900,64900,63300,5521,353682000,548113344000,8564271,STK
4,000080,하이트진로,필수소비재,"식품,음료,담배",KR7000080002,하이트진로,KOSPI,,19120,1,50,0.26,19090,19160,18980,67602,1289245210,1340954642320,70133611,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,489480,키움제11호스팩,금융,다각화된금융,KR7489480004,키움제11호스팩,KOSDAQ,SPAC(소속부없음),1965,2,-8,-0.41,1973,1973,1957,4750,9312657,12969000000,6600000,KSQ
2511,489730,디비금융제13호스팩,금융,다각화된금융,KR7489730002,디비금융제13호스팩,KOSDAQ,SPAC(소속부없음),1961,1,1,0.05,1961,1964,1957,10448,20480785,12417052000,6332000,KSQ
2512,489790,한화비전,산업재,자본재,KR7489790006,한화비전,KOSPI,,34150,2,-3950,-10.37,37800,37950,33800,2523305,89157988900,1724178518500,50488390,STK
2513,492220,KB제31호스팩,금융,다각화된금융,KR7492220009,KB제31호스팩,KOSDAQ,SPAC(소속부없음),1951,1,5,0.26,1946,1952,1942,54620,106390046,12115710000,6210000,KSQ


In [16]:
# 열 이름 변경
df_merged = df_merged.rename(columns={'Name_x': 'Name'})

# 필요한 열만 선택
df_merged = df_merged[['Code', 'Name', 'Ls', 'Ms', 'Marcap', 'Close']]

In [17]:
df_merged

,Code,Name,Ls,Ms,Marcap,Close
0,000020,동화약품,건강관리,제약과생물공학,174851002200,6260
1,000040,KR모터스,경기관련소비재,자동차와부품,27059790600,450
2,000050,경방,경기관련소비재,내구소비재와의류,177650949600,6480
3,000070,삼양홀딩스,소재,소재,548113344000,64000
4,000080,하이트진로,필수소비재,"식품,음료,담배",1340954642320,19120
...,...,...,...,...,...,...
2510,489480,키움제11호스팩,금융,다각화된금융,12969000000,1965
2511,489730,디비금융제13호스팩,금융,다각화된금융,12417052000,1961
2512,489790,한화비전,산업재,자본재,1724178518500,34150
2513,492220,KB제31호스팩,금융,다각화된금융,12115710000,1951


valueline 가치지표 추출

In [23]:
# 항목 추출 함수
def get_financial_data_parallel(code):
    url = f"https://www.valueline.co.kr/finance/investment/{code}"
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        financial_data = {
            "종목진단":None,
            "EPS": None,
            "BPS": None,
            "PER": None,
            "PBR": None,
            "PSR": None,
            "PCR": None,
            "EV/EBITDA": None,
            "영업이익률": None,
            "순이익률": None,
            "ROE": None,
            "ROA": None,
            "ROIC": None,
            "부채비율":None
        }
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # 종목진단 점수 추출
            diagnosis_element = soup.select_one("span.summary_txtShow strong")
            if diagnosis_element:
                diagnosis_score = diagnosis_element.get_text(strip=True).replace("점", "")
                print(f"종목진단 점수: {diagnosis_score}")
            else:
                print("종목진단 점수를 찾을 수 없습니다.")

            financial_data["종목진단"] = diagnosis_score

            # 테이블 추출
            table = soup.find("table", class_="table fix_table data_txtR txtInt")
            if table:
                rows = table.find_all("tr")
                for row in rows:
                    header = row.find("th", class_="depth_1 fix txt_L")
                    if not header:
                        continue
                    header_text = header.get_text(strip=True)
                    value_td = row.find("td")
                    if value_td:
                        value = value_td.get_text(strip=True)
                        if header_text == "주당순이익 EPS(원)":
                            financial_data["EPS"] = value
                        elif header_text == "주당순자산 BPS(원)":
                            financial_data["BPS"] = value
                        elif header_text == "주가수익배수 PER(배)":
                            financial_data["PER"] = value
                        elif header_text == "주가순자산배수 PBR(배)":
                            financial_data["PBR"] = value
                        elif header_text == "주가매출액배수 PSR(배)":
                            financial_data["PSR"] = value
                        elif header_text == "주가현금흐름배수 PCR(배)":
                            financial_data["PCR"] = value
                        elif header_text == "EV/EBITDA(배)":
                            financial_data["EV/EBITDA"] = value
                        elif header_text == "영업이익률(%)":
                            financial_data["영업이익률"] = value
                        elif header_text == "순이익률(%)":
                            financial_data["순이익률"] = value
                        elif header_text == "자기자본이익률 ROE(%)":
                            financial_data["ROE"] = value
                        elif header_text == "총자산이익률 ROA(%)":
                            financial_data["ROA"] = value
                        elif header_text == "투하자본이익률ROIC(%)":
                            financial_data["ROIC"] = value
                        elif header_text == "부채비율(%)":
                            financial_data["부채비율"] = value
        return code, financial_data
    except Exception as e:
        print(f"Error fetching data for code {code}: {e}")
        return code, None

# 병렬 처리
codes = df_merged["Code"].tolist()
financial_data_list = []

with ThreadPoolExecutor(max_workers=10) as executor:  # 동시 요청 수 제한 (10개)
    results = executor.map(get_financial_data_parallel, codes)

    for code, data in results:
        if data:
            financial_data_list.append((code, data))

# 데이터를 DataFrame에 추가
for code, data in financial_data_list:
    for key, value in data.items():
        df_merged.loc[df_merged["Code"] == code, key] = value

# 결과 출력
print(df_merged)

종목진단 점수: 42
종목진단 점수: 41
종목진단 점수: 27
종목진단 점수: 45
종목진단 점수: 27
종목진단 점수: 8
종목진단 점수: 55
종목진단 점수: 40
종목진단 점수: 42
종목진단 점수: 46
종목진단 점수: 36
종목진단 점수: 48
종목진단 점수: 26
종목진단 점수: 66
종목진단 점수: 84
종목진단 점수: 44
종목진단 점수: 42
종목진단 점수: 67
종목진단 점수: 52
종목진단 점수: 36
종목진단 점수: 32
종목진단 점수: 45
종목진단 점수: 40
종목진단 점수: 51
종목진단 점수: 18
종목진단 점수: 53
종목진단 점수: 60
종목진단 점수: 48
종목진단 점수: 6
종목진단 점수: 77
종목진단 점수: 56
종목진단 점수: 40
종목진단 점수: 29
종목진단 점수: 36
종목진단 점수: 35
종목진단 점수: 48
종목진단 점수: 48
종목진단 점수: 61
종목진단 점수: 49
종목진단 점수: 15
종목진단 점수: 36
종목진단 점수: 30
종목진단 점수: 49
종목진단 점수: 38
종목진단 점수: 75
종목진단 점수: 30
종목진단 점수: 36
종목진단 점수: 26
종목진단 점수: 60
종목진단 점수: 44
종목진단 점수: 34
종목진단 점수: 62
종목진단 점수: 17
종목진단 점수: 7
종목진단 점수: 47종목진단 점수: 35

종목진단 점수: 58
종목진단 점수: 50
종목진단 점수: 45
종목진단 점수: 61
종목진단 점수: 58
종목진단 점수: 26
종목진단 점수: 19
종목진단 점수: 54
종목진단 점수: 48
종목진단 점수: 38
종목진단 점수: 39
종목진단 점수: 59
종목진단 점수: 49
종목진단 점수: 36종목진단 점수: 24

종목진단 점수: 39
종목진단 점수: 4
종목진단 점수: 48
종목진단 점수: 44
종목진단 점수: 34
종목진단 점수: 24
종목진단 점수: 19
종목진단 점수: 43
종목진단 점수: 31
종목진단 점수: 47
종목진단 점수: 55
종목진단 점수: 37
종목진단 점수:

In [24]:
df_merged

,Code,Name,Ls,Ms,Marcap,Close,종목진단,EPS,BPS,PER,PBR,PSR,PCR,EV/EBITDA,영업이익률,순이익률,ROE,ROA,ROIC,부채비율
0,000020,동화약품,건강관리,제약과생물공학,174851002200,6260,42,275,"13,455",28.25,0.58,0.51,-24.23,9.46,2.10,1.79,2.04,1.31,0.98,48.52
1,000040,KR모터스,경기관련소비재,자동차와부품,27059790600,450,8,-262,519,-3.12,1.08,0.49,-2.37,-113.15,-24.24,-15.60,-34.59,-7.33,-4.10,377.26
2,000050,경방,경기관련소비재,내구소비재와의류,177650949600,6480,45,278,"27,173",21.98,0.23,0.42,3.41,5.45,8.26,1.93,1.02,0.64,2.02,60.30
3,000070,삼양홀딩스,소재,소재,548113344000,64000,46,"5,448","233,993",13.81,0.32,0.19,2.68,7.11,4.16,1.40,2.33,0.93,3.26,107.45
4,000080,하이트진로,필수소비재,"식품,음료,담배",1340954642320,19120,41,"1,535","16,155",13.05,1.24,0.55,4.06,6.29,8.36,4.22,9.50,3.18,6.56,196.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,489480,키움제11호스팩,금융,다각화된금융,12969000000,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2511,489730,디비금융제13호스팩,금융,다각화된금융,12417052000,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2512,489790,한화비전,산업재,자본재,1724178518500,34150,20,N/A,"15,618",-142.75,2.13,0.00,0.00,0.00,0.00,0.00,-1.49,-0.74,N/A,101.58
2513,492220,KB제31호스팩,금융,다각화된금융,12115710000,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


RIM value위한 할인율 산출 V= BPS+(ROE-r)*BPS/r

In [25]:
#ChromeDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

data = []  # 데이터 저장을 위한 리스트 생성

try:
    # Kofia Bond 웹사이트로 이동
    url = 'https://www.kofiabond.or.kr/websquare/websquare.html?w2xPath=/xml/Com/Common_TabMnuDsp.xml&divisionId=MBIS01070010000000&serviceId=BIS0100100280&topMenuIndex=7&w2xHome=/xml/&w2xDocumentRoot=#tabContents1_contents_tabs2_bridge'
    driver.get(url)

    # 메인 콘텐츠 iframe으로 전환 (tabContents1_contents_tabs1_body)
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "tabContents1_contents_tabs1_body")))

    # 기본 콘텐츠로 돌아가 "기간별" 탭 선택
    driver.switch_to.default_content()
    period_tab = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "tabContents1_tab_tabs2"))
    )
    period_tab.click()

    # "기간별" 탭과 관련된 iframe으로 전환 (tabContents1_contents_tabs2_body)
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "tabContents1_contents_tabs2_body")))

    # 시작 날짜 및 종료 날짜 설정
    start_date = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "schSstandardDt_input")))
    start_date.clear()
    start_date.send_keys(date)  # 시작 날짜"20241227"
 
    end_date = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "schEstandardDt_input")))
    end_date.clear()
    end_date.send_keys(date)  # 종료 날짜"20241227"

    # 드롭다운에서 "회사채 I(공모사채) /무보증 /BBB-" 선택
    dropdown_main = Select(WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "selectBndTyp1_input_0"))
    ))
    dropdown_main.select_by_visible_text("회사채 I(공모사채) /무보증 /BBB-")

    # 인접 드롭다운에서 "3년" 선택
    dropdown_duration = Select(WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "selectTrm1_input_0"))
    ))
    dropdown_duration.select_by_visible_text("3년")

    # 각 체크박스 선택
    checkbox_ids = [
        "checkbox1_input_0",
        "checkbox2_input_0",
        "checkbox3_input_0",
        "checkbox4_input_0",
        "checkbox5_input_0"
    ]
    for checkbox_id in checkbox_ids:
        checkbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, checkbox_id)))
        if not checkbox.is_selected():
            checkbox.click()

    # "조회" 버튼 클릭
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "image1"))
    )
    search_button.click()

    # 결과가 로드될 때까지 대기
    time.sleep(3)

    # 동적 스크롤 실행 및 데이터 수집
    scrollable_div = driver.find_element(By.CSS_SELECTOR, "#grdMain2_scrollY_div")
    last_height = 0

    while True:
        # 현재 데이터 수집
        table_rows = driver.find_elements(By.CSS_SELECTOR, "#grdMain2_body_tbody tr")

        for row in table_rows:
            try:
                date = row.find_element(By.CSS_SELECTOR, "td[col_id='standardDt']").text
                value = row.find_element(By.CSS_SELECTOR, "td[col_id='val6']").text
                if {"Date": date, "Value": value} not in data:  # 중복 데이터 방지
                    data.append({"Date": date, "Value": value})
            except Exception as e:
                print(f"해당 행에서 요소를 찾을 수 없습니다: {e}")
                continue

        # 스크롤 다운
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
        time.sleep(1)  # 데이터 로드 시간 확보

        # 새로운 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollTop;", scrollable_div)
        if new_height == last_height:
            break  # 스크롤이 더 이상 진행되지 않으면 종료
        last_height = new_height

    # DataFrame으로 변환
    df_cb3y = pd.DataFrame(data)
    df_cb3y["Value"] = pd.to_numeric(df_cb3y["Value"], errors='coerce')
    df_cb3y['Date'] = df_cb3y['Date'].str.replace('/', '-')
    df_cb3y.rename(columns={"Value": "Yield"}, inplace=True)

    ld_cb3y =  df_cb3y.loc[0, 'Yield']
    # 결과 확인
    ld_discount_rate = max(ld_cb3y/100, 0.08)
    print(ld_discount_rate)
except Exception as e:
    print("오류 발생:", str(e))

finally:
    driver.quit()

0.09023999999999999


In [26]:
df_merged

,Code,Name,Ls,Ms,Marcap,Close,종목진단,EPS,BPS,PER,PBR,PSR,PCR,EV/EBITDA,영업이익률,순이익률,ROE,ROA,ROIC,부채비율
0,000020,동화약품,건강관리,제약과생물공학,174851002200,6260,42,275,"13,455",28.25,0.58,0.51,-24.23,9.46,2.10,1.79,2.04,1.31,0.98,48.52
1,000040,KR모터스,경기관련소비재,자동차와부품,27059790600,450,8,-262,519,-3.12,1.08,0.49,-2.37,-113.15,-24.24,-15.60,-34.59,-7.33,-4.10,377.26
2,000050,경방,경기관련소비재,내구소비재와의류,177650949600,6480,45,278,"27,173",21.98,0.23,0.42,3.41,5.45,8.26,1.93,1.02,0.64,2.02,60.30
3,000070,삼양홀딩스,소재,소재,548113344000,64000,46,"5,448","233,993",13.81,0.32,0.19,2.68,7.11,4.16,1.40,2.33,0.93,3.26,107.45
4,000080,하이트진로,필수소비재,"식품,음료,담배",1340954642320,19120,41,"1,535","16,155",13.05,1.24,0.55,4.06,6.29,8.36,4.22,9.50,3.18,6.56,196.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,489480,키움제11호스팩,금융,다각화된금융,12969000000,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2511,489730,디비금융제13호스팩,금융,다각화된금융,12417052000,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2512,489790,한화비전,산업재,자본재,1724178518500,34150,20,N/A,"15,618",-142.75,2.13,0.00,0.00,0.00,0.00,0.00,-1.49,-0.74,N/A,101.58
2513,492220,KB제31호스팩,금융,다각화된금융,12115710000,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# 변환 대상 컬럼 리스트
columns_to_convert = ['Close', 'EPS', '종목진단', 'BPS', 'PER', 'PBR', 'PSR', 'PCR', 'EV/EBITDA', 
                      '영업이익률', '순이익률', 'ROE', 'ROA', 'ROIC', '부채비율']

# 각 컬럼을 숫자형으로 변환
for column in columns_to_convert:
    # 먼저 ','를 제거하고 'N/A'를 NaN으로 변경
    df_merged[column] = df_merged[column].replace({',': '', 'N/A': None}, regex=True)
    
    # 숫자형으로 변환 (NaN이 있으면 해당 값을 NaN으로 처리)
    df_merged[column] = pd.to_numeric(df_merged[column], errors='coerce')

# 결과 확인
print(df_merged.dtypes)

Code          object
Name          object
Ls            object
Ms            object
Marcap         int64
Close          int64
종목진단         float64
EPS          float64
BPS          float64
PER          float64
PBR          float64
PSR          float64
PCR          float64
EV/EBITDA    float64
영업이익률        float64
순이익률         float64
ROE          float64
ROA          float64
ROIC         float64
부채비율         float64
dtype: object


In [28]:
df_merged

,Code,Name,Ls,Ms,Marcap,Close,종목진단,EPS,BPS,PER,PBR,PSR,PCR,EV/EBITDA,영업이익률,순이익률,ROE,ROA,ROIC,부채비율
0,000020,동화약품,건강관리,제약과생물공학,174851002200,6260,42.0,275.0,13455.0,28.25,0.58,0.51,-24.23,9.46,2.10,1.79,2.04,1.31,0.98,48.52
1,000040,KR모터스,경기관련소비재,자동차와부품,27059790600,450,8.0,-262.0,519.0,-3.12,1.08,0.49,-2.37,-113.15,-24.24,-15.60,-34.59,-7.33,-4.10,377.26
2,000050,경방,경기관련소비재,내구소비재와의류,177650949600,6480,45.0,278.0,27173.0,21.98,0.23,0.42,3.41,5.45,8.26,1.93,1.02,0.64,2.02,60.30
3,000070,삼양홀딩스,소재,소재,548113344000,64000,46.0,5448.0,233993.0,13.81,0.32,0.19,2.68,7.11,4.16,1.40,2.33,0.93,3.26,107.45
4,000080,하이트진로,필수소비재,"식품,음료,담배",1340954642320,19120,41.0,1535.0,16155.0,13.05,1.24,0.55,4.06,6.29,8.36,4.22,9.50,3.18,6.56,196.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,489480,키움제11호스팩,금융,다각화된금융,12969000000,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2511,489730,디비금융제13호스팩,금융,다각화된금융,12417052000,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2512,489790,한화비전,산업재,자본재,1724178518500,34150,20.0,NaN,15618.0,-142.75,2.13,0.00,0.00,0.00,0.00,0.00,-1.49,-0.74,NaN,101.58
2513,492220,KB제31호스팩,금융,다각화된금융,12115710000,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# DataFrame 복사본 생성 후 작업
df_merged = df_merged.copy()

# 공식에 따라 'value' 계산
df_merged['Value'] = df_merged['BPS'] + df_merged['BPS'] * ((df_merged['ROE'] / 100 - ld_discount_rate) / ld_discount_rate)
df_merged['Value/Close'] = df_merged['Value']/df_merged['Close']

In [43]:
# DataFrame을 복사하고, 열 이름 변경
df_merged_sorted = df_merged.sort_values(by="Marcap", ascending=False).reset_index(drop=True)

ls_file = 'wics_data_'+date2+'.xlsx'
# 열 이름 매핑
column_mapping = {
    "Code": "코드",
    "Name": "종목",
    "Ls": "WICS 대분류",
    "Ms": "WICS 중분류",
    "Close": "Close",   
    "EPS": "EPS",
    "BPS": "BPS",
    "PER": "PER",
    "PBR": "PBR",
    "PSR": "PSR",
    "PCR": "PCR",
    "EV/EBITDA": "EV/EBITDA",
    "영업이익률": "영업이익률",
    "순이익률": "순이익률",
    "ROE": "ROE",
    "ROA": "ROA",
    "ROIC": "ROIC",
    "부채비율": "부채비율",    
    "Value": "Value"
}

# 열 이름 변경
df_merged_sorted.rename(columns=column_mapping, inplace=True)

# No 열 추가
df_merged_sorted.insert(0, "No", range(1, len(df_merged_sorted) + 1))

# 결과를 xlsx로 저장
df_merged_sorted.to_excel(ls_file, index=False)